In [4]:

import pandas as pd
import numpy as np


#     Read in bank_marketing.csv as a pandas DataFrame.#  
bank = pd.read_csv('bank_marketing.csv')

#  Rename the column "client_id" to "id" in client (leave as-is in the other subsets)
client = bank[['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'housing', 'loan']]
client.rename(columns={'client_id': 'id'}, inplace=True)
#  Rename "duration" to "contact_duration", "previous" to "previous_campaign_contacts", # # "y" to "campaign_outcome", "poutcome" to "previous_outcome", and "campaign" to "number_contacts" in campaign
campaign = bank[['client_id', 'contact', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y']]
campaign.rename(columns={'duration': 'contact_duration', 'previous': 'previous_campaign_contacts', 'y': 'campaign_outcome', 'poutcome': 'previous_outcome', 'campaign': 'number_contacts'}, inplace=True)
#  and "euribor3m" to "euribor_three_months" and "nr_employed" to "number_employed" in economics.
economics = bank[['client_id', 'emp_var_rate', 'cons_price_idx', 'euribor3m', 'nr_employed']]
economics.rename(columns={'euribor3m': 'euribor_three_months', 'nr_employed': 'number_employed'}, inplace=True)

#  Clean the "education" column, changing "." to "_" and "unknown" to NumPy's null values.
client['education'] = client['education'].str.replace('.', '_')
client['education'] = client['education'].replace('unknown', np.nan)
#     Remove periods from the "job" column.
client['job'] = client['job'].str.replace('.', '')

#     Convert "success" and "failure" in the "previous_outcome" and "campaign_outcome" columns to binary (1 or 0), along with the changing "nonexistent" to NumPy's null values in "previous_outcome".
campaign['previous_outcome'] = campaign['previous_outcome'].replace({'success': 1, 'failure': 0, 'nonexistent': np.nan})
campaign['campaign_outcome'] = campaign['campaign_outcome'].replace({'success': 1, 'failure': 0})
#     Add a column called campaign_id in campaign, where all rows have a value of 1.
campaign['campaign_id'] = 1
#     Create a datetime column called last_contact_date, in the format of "year-month-day", where the year is 2022, and the month and day values are taken from the "month" and "day" columns.
# campaign['last_contact_date'] = pd.to_datetime(campaign['month'].astype(str) + '-' + campaign['day'].astype(str) + '-2022')
campaign['last_contact_date'] = pd.to_datetime('2022-' + bank['month'].astype(str) + '-' + bank['day'].astype(str))

#     Remove any redundant data that might have been used to create new columns, ensuring the columns in each subset of the data match the table displayed in the notebook.
#campaign.dtypes
#client.dtypes
#economics.dtypes

#     Save the three DataFrames to csv files without an index as client.csv, campaign.csv, and economics.csv respectively.

client.to_csv('client.csv', index=False)
campaign.to_csv('campaign.csv', index=False)
economics.to_csv('economics.csv', index=False)

#     Create a variable called client_table, containing SQL code as a string to create a table called client using values from client.csv.
#     Create a variable called campaign_table, containing SQL code as a string to create a table called campaign using values from campaign.csv.
#     Create a variable called economics_table, containing SQL code as a string to create a table called economics using values from economics.csv.

### SANITY CHECK! datatypes - some t.ex previous outcome - they're strings in the dataframe, but wb bools in table _ will they import correctly? Make sure!

#     In client, campaign, and economic, ensure the final line copies the data from their respective csv files using the following code snippet:
# \copy table_name from 'file_name.csv' DELIMITER ',' CSV HEADER


client_table = """CREATE TABLE client (
id SERIAL PRIMARY KEY not null,
age INT,
job TEXT,
marital TEXT,
education TEXT,
credit_default BOOLEAN,
housing BOOLEAN,
loan BOOLEAN
)
\COPY client FROM 'client.csv' DELIMITER ',' CSV HEADER
;"""

campaign_table = """CREATE TABLE campaign (
campaign_id SERIAL PRIMARY KEY not null,
client_id SERIAL FOREIGN KEY REFERENCES client(id),
number_contacts INT,
contact_duration INT,
pdays INT,
previous_campaign_contacts INT,
previous_outcome BOOLEAN,
campaign_outcome BOOLEAN,
last_contact_date DATE
)
\COPY campaign FROM 'campaign.csv' DELIMITER ',' CSV HEADER
;"""

economics_table = """CREATE TABLE economics (
client_id SERIAL FOREIGN KEY REFERENCES client(id),
emp_var_rate FLOAT,
cons_price_idx FLOAT,
euribor_three_months FLOAT,
number_employed FLOAT
)
\COPY economics FROM 'economics.csv' DELIMITER ',' CSV HEADER
;"""

# Ah ok so DataCamp's internal checkers do NOT check to see if a foreign key is present got ittt




C:\Users\Potato\AppData\Local\Temp\ipykernel_16200\1462572796.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  client.rename(columns={'client_id': 'id'}, inplace=True)
C:\Users\Potato\AppData\Local\Temp\ipykernel_16200\1462572796.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign.rename(columns={'duration': 'contact_duration', 'previous': 'previous_campaign_contacts', 'y': 'campaign_outcome', 'poutcome': 'previous_outcome', 'campaign': 'number_contacts'}, inplace=True)
C:\Users\Potato\AppData\Local\Temp\ipykernel_16200\1462572796.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

In [5]:
SELECT * FROM economics_table LIMIT 10;
SELECT * FROM campaign_table LIMIT 10;
SELECT * FROM client_table LIMIT 10;



SyntaxError: invalid syntax. Perhaps you forgot a comma? (3662642423.py, line 1)